# Enter the dataset paths , the percentage for the data split and the parameters to train the model

### If you want to download the predefined dataset follow the next 2 steps or else you can skip that

In [ ]:
!python3 tools/misc/download_dataset.py --dataset-name cat --save-dir ./data/cat --unzip --delete

In [ ]:
#### dataset_root is the path , where images and the annotation file consists 
dataset_root=input('Enter the dataset root path')

# annotation_path , where there is a single file annotation in the coco (json) format 
annotation_path=input('Enter the annotation path ,(json file)')

# consists of images for training
train_images=input('Enter the train images path ,(excluding the dataset_root path )')

# In the above there is single annotation file , which consists of whole data annotations .
# The percentages are given to seperate the data into 3 parts (train , test and validation )
train_percentage=input('Enter the percentage to divide the train dataset . Eg:0.8')
test_percentage= input('Enter the percentage to divide the test dataset . Eg:0.1')
val_percentage= input('Enter the percentage to divide the validation dataset . Eg:0.1')

# enter the epochs , upto how much epochs the models needs to train 
MAX_EPOCHS = input('Enter the MAX_EPOCHS ')

# BATCH_SIZE means  "The number of training examples utilized in one iteration".
BATCH_SIZE = input('Enter the BATCH_SIZE')


# select the model , need to do be train

In [ ]:
import sys

# Dictionary of model types and their corresponding options
model_types = {
    "YOLOX": ["YOLOX_s", "YOLOX_m", "YOLOX_l", "YOLOX_x","YOLOX_Darknet53","YOLOX_Nano","YOLOX_Tiny"],
}

# Variable to store user's choice
selected_model_type = "YOLOX"
selected_model_variant = None

# Print model variants for the selected model type
print(f"Please select a {selected_model_type} model variant:")
for i, model_variant in enumerate(model_types[selected_model_type]):
    print(f"{i}. {model_variant}")

# Get input for model variant
while True:
    model_variant_choice = input("Enter choice (0-{}): ".format(len(model_types[selected_model_type]) - 1))
    try:
        model_variant_choice = int(model_variant_choice)
        if 0 <= model_variant_choice < len(model_types[selected_model_type]):
            selected_model_variant = model_types[selected_model_type][model_variant_choice]
            break
    except ValueError:
        pass

    print("Invalid choice, please try again")

print(f"You selected: {selected_model_variant}")


You selected: YOLOX_l


# split the annotation file into train , test, validation and data augmentation

### If there is single annotation file , that can divide into train , test and validation (coco format)

In [ ]:
pwd

In [ ]:
%cd /root/workspace/mmyolo

In [ ]:
# !pip install sklearn
!pip install funcy
!pip install argparse
!pip install scikit-learn

In [ ]:
!python cocosplit_train_test_valid.py \
    --annotations "{annotation_path}" \
    --train_ratio {train_percentage} \
    --valid_ratio {val_percentage} \
    --test_ratio {test_percentage} \
    --trainJson_name train.json \
    --validJson_name val.json \
    --testJson_name test.json

In [ ]:
import os , shutil
# dataset_root = '/home/viso/datasets/viso_datasets/construction-demo'

parent_directory = os.path.dirname(annotation_path)
shutil.copy('/root/workspace/mmyolo/test.json',parent_directory)
shutil.copy('/root/workspace/mmyolo/val.json',parent_directory)
shutil.copy('/root/workspace/mmyolo/train.json',parent_directory)

train_json = os.path.join(parent_directory, 'train.json')
val_json = os.path.join(parent_directory, 'val.json')
test_json = os.path.join(parent_directory, 'test.json')

# Remove the dataset_root prefix using os.path.relpath
train_json_relpath = os.path.relpath(train_json, dataset_root)
val_json_relpath = os.path.relpath(val_json, dataset_root)
test_json_relpath = os.path.relpath(test_json, dataset_root)

print(train_json_relpath, val_json_relpath, test_json_relpath)
print(parent_directory)

In [ ]:
import os
import shutil

# Define the source directory
source_directory = '/root/workspace/mmyolo/'
destination_directory = os.path.dirname(annotation_path)
print(destination_directory)
# Define the destination directory
# destination_directory = '/home/viso/sample_dataset/annotations'

# Copy the files from the source directory to the destination directory
shutil.copy(os.path.join(source_directory, 'train.json'), destination_directory)
shutil.copy(os.path.join(source_directory, 'val.json'), destination_directory)
shutil.copy(os.path.join(source_directory, 'test.json'), destination_directory)

# Print the relative paths of the copied files with respect to the destination directory
train_json_relpath=os.path.basename(os.path.join(destination_directory, 'train.json'))
print(train_json_relpath)
val_json_relpath=os.path.basename(os.path.join(destination_directory, 'val.json'))
test_json_relpath=os.path.basename(os.path.join(destination_directory, 'test.json'))

# Generating info for the config based on the annotation file 

In [ ]:
import json

with open(annotation_path, 'r') as f:
    data = json.load(f)

classes = [c['name'] for c in data['categories']]
NUM_CLASSES = len(classes)
palette = []

for i, c in enumerate(classes):
    r = (i * 50) % 255
    g = (i * 100) % 255
    b = (i * 150) % 255
    palette.append((r, g, b))

metainfo = {
    'classes': classes,
    'palette': palette
}
class_names=metainfo['classes']
print("Class Names:", class_names)
print("Number of Classes:", NUM_CLASSES)

In [ ]:
# Define the variable containing the class names
# Write the COCO_CLASSES tuple using the class_names variable
with open(f"{dataset_root}/coco_classes.py", "w") as f:
    for class_name in class_names:
        f.write(f'"{class_name}"\n')


# Installing YOLOX Dependencies

In [ ]:
%cd /root/workspace/
!pwd
!git clone https://github.com/Megvii-BaseDetection/YOLOX

In [ ]:
%cd /root/workspace/YOLOX
!pip3 install -U pip && pip3 install -r requirements.txt
!pip3 install -v -e .  
!pip uninstall -y torch torchvision torchaudio
# May need to change in the future if Colab no longer uses CUDA 11.0
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
     

In [ ]:
%cd /root/workspace/YOLOX
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

!pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
     

In [ ]:
selected_model_variant_lower=selected_model_variant.lower()

In [ ]:
selected_model_variant_lower

In [ ]:
train_json_relpath

In [ ]:
# Check if the selected model type is 'YOLOx'
if selected_model_type == 'YOLOX':
    # Provide the path to the user for modification
    dummy_config = f'/root/workspace/YOLOX/exps/default/{selected_model_variant_lower}.py'
    custom_config_file_path_base = f'/root/workspace/YOLOX/exps/example/custom/{selected_model_variant_lower}.py'  # Set your desired path

    # Load the YOLOx config file
    with open(dummy_config, 'r') as f:
        cfg_content = f.read()

    # Split the content of the config file into lines
    cfg_lines = cfg_content.split('\n')

    # Find the index of the line where `self.exp_name` is defined
    exp_name_line_index = cfg_lines.index('        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]')

    # Append specific lines with new values after `self.exp_name`
    cfg_lines.insert(exp_name_line_index + 1, f'        self.data_dir = "{dataset_root}"')
    cfg_lines.insert(exp_name_line_index + 2, f'        self.train_ann = "{train_json_relpath}"')
    cfg_lines.insert(exp_name_line_index + 3, f'        self.val_ann = "{val_json_relpath}"')
    cfg_lines.insert(exp_name_line_index + 4, f'        self.num_classes = {NUM_CLASSES}')
    cfg_lines.insert(exp_name_line_index + 5, f'        self.max_epoch = {MAX_EPOCHS}')
    cfg_lines.insert(exp_name_line_index + 6, f'        self.data_num_workers = 4')
    cfg_lines.insert(exp_name_line_index + 7, f'        self.eval_interval = 1')
    cfg_lines.insert(exp_name_line_index + 8, f'        self.batch_size = {BATCH_SIZE}')

    # Join the lines back into a single string
    cfg_content = '\n'.join(cfg_lines)

    # Save the modified content to the new file
    with open(custom_config_file_path_base, 'w') as custom_cfg_file:
        custom_cfg_file.write(cfg_content)

    print(f"Custom config file has been created at {custom_config_file_path_base}.")
    print("dummy_config:", dummy_config)
    print("custom_config_file_path_base:", custom_config_file_path_base)


In [ ]:
# # train_json_relpath = "'annotations/train.json'"
# train_json_relpath = train_json_relpath.replace("annotations/", "")
# val_json_relpath = val_json_relpath.replace("annotations/", "")
# # train_json_relpath = train_json_relpath.replace("annotations/", "")

print(train_json_relpath)


In [ ]:
cd /root/workspace/YOLOX

In [ ]:
!pip install loguru
!pip install tensorboard

In [ ]:
%cd /root/workspace/YOLOX
!wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/{selected_model_variant_lower}.pth

In [ ]:
selected_model_variant_lower

In [ ]:
train_images

In [ ]:
import fileinput

# Define the new value for train_path
# train_path = "your_desired_train_path"

# File to modify
file_path = "/root/workspace/YOLOX/yolox/data/datasets/coco.py"

# Read the file and replace the desired line
with fileinput.FileInput(file_path, inplace=True, backup=".bak") as file:
    for line in file:
        print(line.replace('name="train2017"', f'name="{train_images}"'), end='')


In [ ]:
import subprocess
import sys

# Get the current Python path
PYTHONPATH = ":".join(sys.path)

# Assuming config_file_path is something like "yolox/config_file.py"
# config_file_path = config_file_path.split('/')[-1]
command_line = f"export PYTHONPATH='{PYTHONPATH}:/root/workspace/YOLOX/' && python tools/train.py -f {custom_config_file_path_base} -d 1 -b 32 --fp16 -o -c /root/workspace/YOLOX/{selected_model_variant_lower}.pth"

# Run the command
result = subprocess.run(command_line, shell=True)

# Check the result
if result.returncode == 0:
    print("Command executed successfully")
else:
    print(f"Error executing the command. Return code: {result.returncode}")
    print(f"Output: {result.stdout}")
    print(f"Error: {result.stderr}")
